## 读取数据

In [1]:
import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt
import numpy as np
from sklearn import metrics
import seaborn as sns
import matplotlib
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
plt.rcParams['figure.dpi'] = 150 # 修改图片分辨率
plt.rcParams.update({'font.size': 18})
plt.rcParams['font.family'] = 'sans-serif'
plt.rcParams['font.sans-serif'] = ['Helvetica Neue']
import matplotlib.pyplot as plt


In [2]:
# Load the data
data = pd.read_excel('./P0022023-study2.xlsx')

# 删除 'Cluster' 列中等于 3 的行
data= data[data['Label'] != 3]

data
exclusive = ['Label']
data[ exclusive] = data[ exclusive].fillna(-1)
data = data.drop(columns=['ID'])
data # display the first few rows of the processed data

,Subtype I Baseline,Subtype II Baseline,Label,Group
0,0.792498,0.207502,0,0
1,0.144166,0.855834,1,1
2,0.256416,0.743584,1,0
3,0.341691,0.658309,1,0
4,0.216101,0.783899,1,1
...,...,...,...,...
81,0.162009,0.837991,1,0
82,0.873583,0.126417,0,0
83,0.195013,0.804987,1,0
84,0.196676,0.803324,1,0


## 数据不平衡数据

In [3]:
df=data
df[exclusive] = df[exclusive].fillna(-1)
cols=[i for i in df.columns if i not in ['Group']]
X=df[cols]
y=df['Group']

from collections import Counter
print(sorted(Counter(y).items()))
from imblearn.over_sampling import SMOTE, ADASYN

X_resampled, y_resampled = SMOTE(sampling_strategy=0.90, random_state=1).fit_resample(X, y)
# 打印处理之前的类别人数
print("处理之前的类别人数:")
print(y.value_counts())

[(0, 51), (1, 27)]
处理之前的类别人数:
0    51
1    27
Name: Group, dtype: int64


In [4]:
from collections import Counter
print(sorted(Counter(y_resampled).items()))

[(0, 51), (1, 45)]


In [5]:
import pickle
# 保存这些变量到文件：X_resampled，y_resampled和cols_select
with open('data/variables-huafen.pkl', 'wb') as file:
    pickle.dump((X_resampled, y_resampled), file)


# 将数据保存为 Excel 文件
if isinstance(X_resampled, pd.DataFrame) or isinstance(X_resampled, pd.Series):
    X_resampled.to_excel('table/X_resampled.xlsx')

if isinstance(y_resampled, pd.DataFrame) or isinstance(y_resampled, pd.Series):
    y_resampled.to_excel('table/y_resampled.xlsx')

In [6]:
X_resampled

,Subtype I Baseline,Subtype II Baseline,Label
0,0.792498,0.207502,0
1,0.144166,0.855834,1
2,0.256416,0.743584,1
3,0.341691,0.658309,1
4,0.216101,0.783899,1
...,...,...,...
91,0.121390,0.878610,1
92,0.231345,0.768655,1
93,0.162059,0.837941,1
94,0.156096,0.843904,1


### 模拟输入

In [7]:
import pickle
from sklearn.model_selection import train_test_split
exclusive = ['Label']
with open('./data/variables-huafen.pkl', 'rb') as file:
    X_resampled, y_resampled = pickle.load(file)
X , y = X_resampled,y_resampled
X

,Subtype I Baseline,Subtype II Baseline,Label
0,0.792498,0.207502,0
1,0.144166,0.855834,1
2,0.256416,0.743584,1
3,0.341691,0.658309,1
4,0.216101,0.783899,1
...,...,...,...
91,0.121390,0.878610,1
92,0.231345,0.768655,1
93,0.162059,0.837941,1
94,0.156096,0.843904,1


In [8]:
_ = 0 
# 步骤1: 分层随机分割数据集
X_train, X_holdout, y_train, y_holdout = train_test_split(X, y, test_size=0.2, stratify=y, random_state=0)
# # 获取划分后的索引
train_indices = X_train.index
test_indices = X_holdout.index
np.savez('split/indices.npz', train=train_indices, test=test_indices)


In [9]:
import numpy as np

data = np.load('./split/indices.npz')

In [10]:
train_PANSS = X_resampled[exclusive].loc[train_indices]
train_PANSS[(train_PANSS != -1).any(axis=1)]

,Label
49,1
11,0
32,1
56,1
23,1
...,...
66,0
19,0
60,1
52,1


In [11]:
test_PANSS = X_resampled[exclusive].loc[test_indices]
print(len(test_PANSS[(test_PANSS != -1).any(axis=1)]))
test_PANSS[(test_PANSS != -1).any(axis=1)]

20


,Label
89,1
38,1
28,1
71,0
15,1
33,0
95,1
17,1
87,1
5,0


In [12]:
test_indices

Int64Index([89, 38, 28, 71, 15, 33, 95, 17, 87, 5, 65, 76, 74, 44, 82, 13, 39,
            0, 61, 1],
           dtype='int64')

In [13]:
print("预处理全部完成！")

预处理全部完成！
